In [46]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [47]:
from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    classification_report, 
    confusion_matrix, 
    roc_auc_score, 
    precision_score, 
    recall_score,
)
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall

In [48]:
df = pd.read_csv("../../our_analyses/dataset_prepared.csv")
df_test= pd.read_csv("../../our_analyses/dataset_test_prepared.csv")

In [49]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

df=df.drop(['name', 'artists', 'album_name'], axis=1)
df_test=df_test.drop(['name', 'artists', 'album_name'], axis=1)

genre_groups = {
    # Dance/Electronic - Generi che sono comunemente associati alla musica da discoteca e da ballo elettronica.
    'j-dance': 0, 'techno': 0, 'chicago-house': 0, 'breakbeat': 0, 'idm': 0,
    
    # Ambient/Relaxing - Musica rilassante spesso usata per creare un'atmosfera tranquilla, per lo studio o il sonno.
    'happy': 1, 'sleep': 1, 'study': 1, 'disney': 1,
    
    # Global/Traditional - Musica che spesso ha radici culturali tradizionali o regionali.
    'bluegrass': 2, 'forro': 2, 'mandopop': 2, 'iranian': 2, 'indian': 2, 'brazil': 2,
    
    # Metal/Industrial - Generi noti per essere intensi, con sonorità pesanti o industriali.
    'black-metal': 3, 'industrial': 3,
    
    # Pop/World - Musica popolare che può includere influenze internazionali o etniche.
    'j-idol': 4, 'spanish': 4, 'afrobeat': 4,
    

}


df['genre_group'] = df['genre'].map(genre_groups).astype(int)
df_test['genre_group'] = df_test['genre'].map(genre_groups).astype(int)

le = LabelEncoder()
df['explicit'] = le.fit_transform(df['explicit'])
df_test['explicit'] = le.transform(df_test['explicit'])

drop_attributes=[ 'explicit', 'mode', 'time_signature']
df = df.drop(drop_attributes, axis=1)
df_test = df_test.drop(drop_attributes, axis=1)

In [50]:
# Separazione delle features e dei target
X_train = df.drop(['genre_group', 'genre'], axis=1)
y_train = df['genre_group'].values
X_test = df_test.drop(['genre_group', 'genre'], axis=1)
y_test = df_test['genre_group'].values

In [51]:
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)

In [61]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
best_params_well = {
    'criterion': 'entropy',
    'max_depth': 10,
    'min_samples_leaf': 20,
    'min_samples_split': 30
}
# Addestramento del classificatore KNN
clf = DecisionTreeClassifier(**best_params_well, ccp_alpha=0.0004273469893413006)
clf.fit(X_train_norm, y_train)

# Valutazione del classificatore sul set di addestramento
y_train_pred = clf.predict(X_train_norm)
print("Accuracy sul set di addestramento:", accuracy_score(y_train, y_train_pred))

# Valutazione del classificatore sul set di test
y_test_pred = clf.predict(X_test_norm)
print("Accuracy sul set di test:", accuracy_score(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

Accuracy sul set di addestramento: 0.7144
Accuracy sul set di test: 0.63
              precision    recall  f1-score   support

           0       0.62      0.66      0.64      1250
           1       0.66      0.65      0.66      1000
           2       0.69      0.75      0.72      1500
           3       0.56      0.60      0.58       500
           4       0.49      0.34      0.40       750

    accuracy                           0.63      5000
   macro avg       0.60      0.60      0.60      5000
weighted avg       0.62      0.63      0.62      5000



In [58]:
import plotly.graph_objs as go
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Assicurati che Z_train, Z_test, t_train, e t_test siano definiti e che i tuoi dati siano già preelaborati
# Z_train, Z_test, t_train, t_test = ...

# Previsione delle probabilità per il set di test
y_score = clf.predict_proba(X_test_norm)

# Binarizza il vettore delle etichette in un formato "one-vs-all"
y_test_binarize = label_binarize(y_test, classes=np.unique(y_train))

# Calcola ROC curve e AUC per ciascuna classe
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = y_score.shape[1]

# Preparare i dati per il plot
traces = []

# Calcola la ROC curve per ogni classe
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarize[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    traces.append(go.Scatter(x=fpr[i], y=tpr[i], mode='lines', 
                             name=f'Class {i} (AUC = {roc_auc[i]:.2f})'))

# Aggiungi la linea diagonale che rappresenta il caso di "indovinare casualmente"
traces.append(go.Scatter(x=[0, 1], y=[0, 1], mode='lines', name='Chance', 
                         line=dict(dash='dash')))

# Crea il layout del plot
layout = go.Layout(title='Multiclass ROC Curve',
                   xaxis=dict(title='False Positive Rate'),
                   yaxis=dict(title='True Positive Rate'),
                   width=1000,
                   height=800,
                   showlegend=True,
                   legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))

fig = go.Figure(data=traces, layout=layout)
fig.show()
